# 3. Datenkonsistenz

## 1. DataFrames importieren

In [62]:
# Bibliotheken importieren
import pandas as pd
import os

# Hauptpfad definieren
path = r'C:\Users\faink\Desktop\Achievement_4\02_Daten'

# Produkte importieren (aus Rohe_Daten)
df_prods = pd.read_csv(os.path.join(path, 'Rohe_Daten', 'products.csv'))

# Falls orders_wrangled.csv im Ordner Prepared_Daten liegt:
df_ords = pd.read_csv(os.path.join(path, 'Prepared_Daten', 'orders_wrangled.csv'))

## 2. Konsistenzprüfung für df_prods (falls noch nicht gemacht)

### - Fehlende Werte prüfen

In [63]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

Die Überprüfung zeigt, dass fehlende Werte ausschließlich in der Spalte „product_name“ auftreten. 
Da es sich um String-Werte handelt, ist eine Imputation nicht sinnvoll. 
Die betroffenen Zeilen werden daher entfernt.

### - Fehlende Werte entfernen

In [64]:
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

# Kontrolle:
df_prods.shape
df_prods_clean.shape

(49677, 5)

### - Duplikate prüfen

In [65]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups

# Kontrolle:
df_dups.shape

(5, 5)

Es wurden vollständige Duplikate identifiziert. 
Diese stellen redundante Datensätze dar und können die Analyse verfälschen.

### - Duplikate entfernen

In [66]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

# Kontrolle:
df_prods_clean_no_dups.shape

(49672, 5)

## 3. df.describe() auf df_ords anwenden

In [67]:
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


Die Minimal- und Maximalwerte wurden überprüft:

- „order_dow“ liegt zwischen 0 und 6 → plausibel (7 Wochentage).
- „order_hour_of_day“ liegt zwischen 0 und 23 → plausibel (24 Stunden).
- „days_since_prior_order“ enthält keinen negativen Wert → korrekt.

Es sind keine auffälligen Extremwerte erkennbar, die weiter untersucht werden müssten.

# 4. Prüfung auf gemischte Datentypen in df_ords

In [68]:
for col in df_ords.columns:
    first_type = type(df_ords[col].iloc[0])
    weird = df_ords[col].apply(lambda x: type(x) != first_type)
    if weird.any():
        print(col)

order_id
user_id
order_number
order_dow
order_hour_of_day
days_since_prior_order


Es wurden keine Spalten mit gemischten Datentypen identifiziert.
Alle Variablen enthalten konsistente Datentypen.

# 5. Fehlende Werte in df_ords prüfen

In [69]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

Fehlende Werte treten in der Spalte „days_since_prior_order“ auf.
Dies ist plausibel, da Erstbestellungen keinen vorherigen Bestellzeitpunkt haben.

### - Fehlende Werte sinnvoll behandeln

In [70]:
df_ords['days_since_prior_order'] = df_ords['days_since_prior_order'].fillna(0)

# Kontrolle:
df_ords.isnull().sum()

order_id                  0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
dtype: int64

Die fehlenden Werte wurden durch 0 ersetzt, da es sich um Erstbestellungen handelt, bei denen kein vorheriger Bestellzeitpunkt existiert. 
Diese Vorgehensweise ist sachlich begründet und verzerrt die Analyse nicht.

## 6. Duplikate in df_ords prüfen

In [71]:
df_ords_dups = df_ords[df_ords.duplicated()]
df_ords_dups

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


Es wurden vollständige Duplikate geprüft.
Falls vorhanden, stellen diese redundante Bestellungen dar und sollten entfernt werden.

### - Duplikate entfernen

In [72]:
df_ords['days_since_prior_order'] = df_ords['days_since_prior_order'].fillna(0)

# Kontrolle:
df_ords_clean.shape

(3421083, 7)

Die Duplikate wurden entfernt, um redundante Bestelldaten zu vermeiden.
Dadurch wird die Datenqualität verbessert und eine verzerrungsfreie Analyse sichergestellt.

## 7. Finale Exporte

In [73]:
df_prods_clean_no_dups.to_csv(
    os.path.join(path, 'Prepared_Daten', 'products_checked.csv'),
    index=False
)

df_ords_clean.to_csv(
    os.path.join(path, 'Prepared_Daten', 'orders_checked.csv'),
    index=False
)

Im Rahmen dieser Analyse wurden folgende Prüfungen durchgeführt:

- Identifikation und Bereinigung fehlender Werte
- Überprüfung und Entfernung von Duplikaten
- Kontrolle auf gemischte Datentypen
- Plausibilitätsprüfung mittels deskriptiver Statistik

Nach Abschluss der Konsistenzprüfungen stehen bereinigte und analysierbare Datensätze zur Verfügung.